In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.optimize import brentq

$$\begin{aligned}
P_0 = 100mio P_swap_0 + 1mio AAPL_0& + 1mio MSFT_0 + 1mio F + 1mio BAC_0     \\
P_1 = 100mio P_swap_1 + 1mio AAPL_1& + 1mio MSFT_1 + 1mio F_1 + 1mio BAC_1   \\

L = P_1& - P_0  \\

L (\mu , \sigma^2) \\

\mu \approx 100mio \mathbb{E}[\Delta_{swap}] + 1mio \mathbb{E}&[\Delta_{aapl} + \Delta_{msft}  + \Delta_{f}  + \Delta_{bac}]        \\

\sigma^2 \approx \text{Var}(100mio\Delta_{swap}) + \text{Var}(1mio &[\Delta_{aapl} + \Delta_{msft}  + \Delta_{f}  + \Delta_{bac}])
\end{aligned}$$

In [2]:
aapl = pd.read_excel("./data/hist_data.xlsm", sheet_name = 'AAPL')
msft = pd.read_excel("./data/hist_data.xlsm", sheet_name = 'MSFT')
f = pd.read_excel("./data/hist_data.xlsm", sheet_name = 'F')
bac = pd.read_excel("./data/hist_data.xlsm", sheet_name = 'BAC')
sofr_curve = pd.read_excel("./data/hist_data.xlsm", sheet_name = 'SofrCurve')

# Extract "T" as a separate array
T_array = sofr_curve["T"].values
df_transposed = sofr_curve.set_index("T").T

# Rename index to "Date"
df_transposed.index.name = "Date"
df_transposed.reset_index(inplace=True)
col_names = ['Date'] + list(df_transposed.loc[0][1:].values)
df_transposed.columns = col_names
sofr_curve = df_transposed[1:]
sofr_curve['Date'] = pd.to_datetime(sofr_curve['Date'])
sofr_curve.reset_index(inplace=True,drop=True)

/var/folders/h2/r7qn2m9n1zb6y_0q191gdqth0000gn/T/ipykernel_2806/1798135338.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sofr_curve['Date'] = pd.to_datetime(sofr_curve['Date'])


In [12]:
df_stocks = pd.concat([aapl['Adj Close'],
                        msft['Adj Close'],
                        f['Adj Close'],
                        bac['Adj Close']],axis=1)
df_stocks.columns = ['aapl', 'msft', 'f', 'bac']

df_sofr = sofr_curve[sofr_curve.columns[1:]]

In [ ]:
df_stocks.head()

,aapl,msft,f,bac
aapl,351.380323,724.105442,9.918896,-35.916317
msft,724.105442,1622.654311,16.021257,-90.258729
f,9.918896,16.021257,0.958969,0.264916
bac,-35.916317,-90.258729,0.264916,9.617016


In [16]:
df_sofr.head()

,1D,1M,2M,3M,6M,9M,1Y,2Y,3Y,4Y,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
0,0.039191,0.038721,0.03867,0.040536,0.044577,0.046004,0.046449,0.044583,0.042002,0.040318,...,0.037151,0.037057,0.036907,0.036698,0.036433,0.036111,0.034091,0.03235,0.030552,0.028708
1,0.039604,0.039023,0.038886,0.040725,0.044849,0.046448,0.04697,0.045022,0.042344,0.040614,...,0.036802,0.036682,0.036511,0.036287,0.03601,0.035678,0.033645,0.031979,0.030238,0.028478
2,0.039948,0.039286,0.0391,0.040852,0.044884,0.04658,0.047203,0.045496,0.042749,0.040868,...,0.036855,0.036701,0.036498,0.036248,0.035953,0.035613,0.033627,0.031936,0.030292,0.028608
3,0.040389,0.039585,0.03935,0.041154,0.045281,0.047107,0.047894,0.046594,0.043833,0.041825,...,0.037221,0.037069,0.036886,0.036657,0.036372,0.036022,0.033811,0.032134,0.030407,0.028655
4,0.045965,0.042343,0.038795,0.040611,0.045212,0.046752,0.0475,0.046097,0.043385,0.041503,...,0.037687,0.037557,0.03738,0.037152,0.03687,0.036534,0.034424,0.032558,0.030723,0.028933


In [17]:
df_stocks.cov()

,aapl,msft,f,bac
aapl,351.380323,724.105442,9.918896,-35.916317
msft,724.105442,1622.654311,16.021257,-90.258729
f,9.918896,16.021257,0.958969,0.264916
bac,-35.916317,-90.258729,0.264916,9.617016


In [19]:
df_sofr.cov().head()

,1D,1M,2M,3M,6M,9M,1Y,2Y,3Y,4Y,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
1D,0.000022,0.000022,0.000019,0.000016,0.000012,0.000010,0.000008,0.000006,0.000007,0.000008,...,0.000009,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
1M,0.000022,0.000023,0.000021,0.000018,0.000013,0.000011,0.000008,0.000006,0.000007,0.000007,...,0.000009,0.000009,0.000009,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
2M,0.000019,0.000021,0.000019,0.000017,0.000012,0.000010,0.000008,0.000006,0.000006,0.000007,...,0.000008,0.000008,0.000008,0.000008,0.000008,0.000009,0.000009,0.000009,0.000009,0.000009
3M,0.000016,0.000018,0.000017,0.000014,0.000011,0.000009,0.000007,0.000006,0.000006,0.000006,...,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
6M,0.000012,0.000013,0.000012,0.000011,0.000009,0.000008,0.000007,0.000007,0.000007,0.000007,...,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007
